<a href="https://colab.research.google.com/github/ponakilan/vid-anomaly/blob/main/RC_IPAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ! 7z x '/content/drive/MyDrive/Research Credit/IPAD_dataset.zip'

! unzip -q '/content/drive/MyDrive/Research Credit/IPAD_dataset.zip' 'IPAD_dataset/R01/training/frames/*'

In [8]:
import torch
import tensorflow as tf
from transformers import ViTFeatureExtractor, ViTModel
import numpy as np

SEQ_LEN = 10
BATCH_SIZE = 10
TRAIN_DIR = "/content/IPAD_dataset/R01/training/frames"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)  # Move model to GPU


def get_vit_embeddings(image_batch, labels_batch):
    image_batch = image_batch.numpy().astype(np.float32) / 255.0
    image_batch = [img for img in image_batch]

    inputs = feature_extractor(images=image_batch, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embeddings, labels_batch


def replace_labels_with_images(x, y):
    return x, x


dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_DIR,
    batch_size=SEQ_LEN,
    image_size=(224, 224),
    shuffle=False
)

dataset = dataset.map(replace_labels_with_images)

dataset = dataset.map(lambda x, y: tf.py_function(
    func=get_vit_embeddings, inp=[x, y], Tout=(tf.float32, tf.float32))
)

dataset = dataset.batch(BATCH_SIZE)  # (B, N, E)

print(f"Number of batches in the dataset: {len(dataset)}")

Using device: cuda


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Found 7808 files belonging to 34 classes.
Number of batches in the dataset: 79


In [10]:
# Test
embed, image = next(iter(dataset))
embed.shape, image.shape

(TensorShape([10, 10, 768]), TensorShape([10, 10, 224, 224, 3]))